In [1]:
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import LabelEncoder
import pandas as pd
from sklearn.metrics import accuracy_score

In [2]:
df = pd.read_csv('../datasets/user_song_mockfeatures.csv')
df['likes_song'] = (df['affinity_score'] > df['affinity_score'].median()).astype(int)

In [3]:


label_encoder = LabelEncoder()
df['track_id_encoded'] = label_encoder.fit_transform(df['track_id'])


# Perform label encoding on the track_id column
df['track_id_encoded'] = label_encoder.fit_transform(df['track_id'])
# Move the 'track_id_encoded' column to the first position
df = df[['track_id_encoded'] + [col for col in df.columns if col != 'track_id_encoded']]
df[['user_id', 'gender', 'avg_bpm', 'age', 'avg_cadence'] + [col for col in df.columns if col not in ['user_id', 'gender', 'avg_bpm', 'age', 'avg_cadence']]]
df.drop(['user_id'], inplace=True, axis=1)
df.head()


,track_id_encoded,track_id,track_name,track_artist,track_popularity,track_album_id,track_album_name,track_album_release_date,playlist_name,playlist_id,...,intensity_zone,intensity_zone_encoded,cluster,affinity_score,gender,avg_bpm,age,avg_cadence,avg_pace,likes_song
0,215,0hv6ku4431kPrjbpBBspw3,Animals - Original Mix,6098,5,0N5QcojgdA6aP9oQ7yYG9A,1148,2013-07-01,gym melbourne house,5BqOZpVEqRDfZScvW1QUyA,...,Zone 4 (Hard),3,10,0.287000,1,165.0,16.0,83.0,6.27,0
1,2080,6uYAPqxP69zfpCbsTHXgiz,Cater 2 U,2462,62,0b6ivSFfDs38MG7aLn9rvO,4221,2004-11-16,hip hop rnb,0Ar0Ng9DlAWZtSPBvOQgOa,...,Zone 3 (Moderate),2,14,0.504277,1,165.0,16.0,83.0,6.27,1
2,521,1gzIbdFnGJ226LTl0Cn2SX,White Wedding - Pt. 1,1083,68,5ebh0MxkqELmk50xHj2b4k,2079,1982-01-01,hard rock cafe,55ybnG2z8rpTIerRfnCsw6,...,Zone 5 (Maximum),4,0,0.395855,1,165.0,16.0,83.0,6.27,0
3,887,2vX03yHcVtFUg2ZMuzhLhS,Outlaw - Original Mix,5252,0,7jV5akfXEXsoNsqrekDCyX,12116,2014-04-21,latin village,3JcJCsvBnRsA6oc7eUYDSB,...,Zone 4 (Hard),3,10,0.285107,1,165.0,16.0,83.0,6.27,0
4,953,38LrhW3eQSt3cK88dkvl5e,Misty Night Cruising,5066,33,24sd1ZTrjjabj0A54XK0RP,11054,1984-12-21,city pop シティーポップ,3j2osvmecEao5nmo9jZ5df,...,Zone 5 (Maximum),4,0,0.292775,1,165.0,16.0,83.0,6.27,0


In [4]:
def df_preprocessing(df):
    # Create an instance of LabelEncoder

    model_df = df.copy()
    model_df = model_df.select_dtypes(exclude=['object'])
    model_df = df[['track_id'] + list(model_df.columns)] #creating a dataset with track_id + model_df columns

    return model_df
        

In [5]:
model_df = df_preprocessing(df)
model_df.drop(['track_album_name', 'intensity_zone_encoded', 'cluster', 'track_id'],inplace=True, axis=1)

/var/folders/gr/q7qsw30x1xqd0zv1tj58r0vw0000gn/T/ipykernel_51684/2579399489.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  model_df.drop(['track_album_name', 'intensity_zone_encoded', 'cluster', 'track_id'],inplace=True, axis=1)


In [6]:
model_df.head()

,track_id_encoded,track_artist,track_popularity,playlist_genre,playlist_subgenre,danceability,energy,key,loudness,speechiness,...,duration_minutes,playlist_name_sentiment,track_name_sentiment,affinity_score,gender,avg_bpm,age,avg_cadence,avg_pace,likes_song
0,215,6098,5,0,18,0.675,0.868,1,-6.360,0.0392,...,5.07,0.000000,0.375,0.287000,1,165.0,16.0,83.0,6.27,0
1,2080,2462,62,3,9,0.605,0.584,11,-7.043,0.1970,...,4.09,0.000000,0.000,0.504277,1,165.0,16.0,83.0,6.27,1
2,521,1083,68,5,7,0.673,0.814,9,-5.411,0.0367,...,4.20,-0.291667,0.000,0.395855,1,165.0,16.0,83.0,6.27,0
3,887,5252,0,1,11,0.557,0.979,6,-4.254,0.0489,...,4.81,0.000000,0.375,0.285107,1,165.0,16.0,83.0,6.27,0
4,953,5066,33,5,0,0.587,0.977,10,-3.589,0.0711,...,4.46,0.000000,0.000,0.292775,1,165.0,16.0,83.0,6.27,0


In [7]:
# Features and target variable
def dividing_df(df):
    X = model_df.drop(['likes_song', 'affinity_score'], axis=1)
    y = model_df['likes_song']
    categorical_features = ['user_id', 'gender', 'playlist_genre', 'playlist_subgenre', 'key', 'track_artist']
    numerical_features = [col for col in model_df.columns if col not in categorical_features and col not in ['likes_song', 'affinity_score'] and col in model_df.columns]
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
    return X_train, X_test, y_train, y_test
 


In [8]:
X_train, X_test, y_train, y_test = dividing_df(model_df)
train_data = lgb.Dataset(X_train, label=y_train)
test_data = lgb.Dataset(X_test, label=y_test, reference=train_data)

In [9]:
params = {
    'boosting_type': 'gbdt',
    'objective': 'regression',
    'metric': 'mse',
    'num_leaves': 31,
    'learning_rate': 0.05,
    'feature_fraction': 0.9,
}

In [10]:
gbm = lgb.train(params, train_data, num_boost_round=100, valid_sets=[test_data])

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000607 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 3454
[LightGBM] [Info] Number of data points in the train set: 2000, number of used features: 23
[LightGBM] [Info] Start training from score 0.499500


In [11]:
y_pred = gbm.predict(X_test, num_iteration=gbm.best_iteration)
y_pred_class = [1 if prob >= 0.5 else 0 for prob in y_pred]


In [12]:
X_test['y_pred'] = y_pred_class
X_test['y_test'] = y_test

In [13]:
mse = mean_squared_error(y_test, y_pred)
print(f"The Mean Squared Error (MSE) on test set: {mse}")

The Mean Squared Error (MSE) on test set: 0.1540997486134686


In [14]:
X_test.head()

,track_id_encoded,track_artist,track_popularity,playlist_genre,playlist_subgenre,danceability,energy,key,loudness,speechiness,...,duration_minutes,playlist_name_sentiment,track_name_sentiment,gender,avg_bpm,age,avg_cadence,avg_pace,y_pred,y_test
892,2236,9251,67,5,2,0.623,0.910,9,-8.448,0.0384,...,3.26,0.166667,0.0,0,73.0,46.0,119.0,7.27,0,0
87,2005,914,4,1,22,0.696,0.664,11,-7.171,0.0445,...,3.27,0.000000,0.0,1,211.0,26.0,73.0,5.91,1,1
1244,161,6098,43,0,1,0.493,0.818,5,-6.252,0.0395,...,3.31,0.000000,0.0,0,79.0,13.0,118.0,5.02,0,0
2267,513,737,68,5,7,0.576,0.906,2,-7.677,0.0571,...,5.40,-0.077652,0.0,1,111.0,40.0,61.0,7.02,1,1
2055,1405,9211,69,5,2,0.619,0.579,4,-9.307,0.0473,...,3.76,0.166667,0.0,1,239.0,27.0,66.0,5.29,1,0


In [15]:
from sklearn.metrics import accuracy_score
# Calculate the accuracy
accuracy = accuracy_score(y_test, y_pred_class)
print(f"The accuracy of the model is: {accuracy}")


The accuracy of the model is: 0.778


In [16]:
feature_importance = gbm.feature_importance()
feature_names = gbm.feature_name()

# Create a dictionary to store the feature importance
feature_importance_dict = dict(zip(feature_names, feature_importance))

# Sort the feature importance dictionary by value in descending order
sorted_feature_importance = sorted(feature_importance_dict.items(), key=lambda x: x[1], reverse=True)

# Print the sorted feature importance
for feature, importance in sorted_feature_importance:
    print(f"{feature}: {importance}")


tempo: 355
loudness: 219
energy: 216
danceability: 170
acousticness: 169
avg_bpm: 158
liveness: 155
duration_minutes: 153
avg_cadence: 149
speechiness: 145
avg_pace: 145
track_artist: 121
valence: 115
playlist_subgenre: 110
age: 109
instrumentalness: 107
track_id_encoded: 106
playlist_genre: 84
track_popularity: 65
key: 55
playlist_name_sentiment: 41
gender: 27
track_name_sentiment: 26


In [17]:
model_df.head()


,track_id_encoded,track_artist,track_popularity,playlist_genre,playlist_subgenre,danceability,energy,key,loudness,speechiness,...,duration_minutes,playlist_name_sentiment,track_name_sentiment,affinity_score,gender,avg_bpm,age,avg_cadence,avg_pace,likes_song
0,215,6098,5,0,18,0.675,0.868,1,-6.360,0.0392,...,5.07,0.000000,0.375,0.287000,1,165.0,16.0,83.0,6.27,0
1,2080,2462,62,3,9,0.605,0.584,11,-7.043,0.1970,...,4.09,0.000000,0.000,0.504277,1,165.0,16.0,83.0,6.27,1
2,521,1083,68,5,7,0.673,0.814,9,-5.411,0.0367,...,4.20,-0.291667,0.000,0.395855,1,165.0,16.0,83.0,6.27,0
3,887,5252,0,1,11,0.557,0.979,6,-4.254,0.0489,...,4.81,0.000000,0.375,0.285107,1,165.0,16.0,83.0,6.27,0
4,953,5066,33,5,0,0.587,0.977,10,-3.589,0.0711,...,4.46,0.000000,0.000,0.292775,1,165.0,16.0,83.0,6.27,0


## Retraining after feature selection

In [18]:
df_2 = model_df.copy()
df_2.drop([col for col, importance in sorted_feature_importance if importance < 90], axis=1, inplace=True)
X_train, X_test, y_train, y_test = dividing_df(df_2)
train_data = lgb.Dataset(X_train, label=y_train)
test_data = lgb.Dataset(X_test, label=y_test, reference=train_data)


In [19]:
gbm = lgb.train(params, train_data, num_boost_round=100, valid_sets=[test_data])
y_pred = gbm.predict(X_test, num_iteration=gbm.best_iteration)
y_pred_class = [1 if prob >= 0.5 else 0 for prob in y_pred]

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000400 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 3458
[LightGBM] [Info] Number of data points in the train set: 2000, number of used features: 23
[LightGBM] [Info] Start training from score 0.499000


In [20]:
mse = mean_squared_error(y_test, y_pred)
print(f"The Mean Squared Error (MSE) on test set: {mse}")
# Calculate the accuracy
accuracy = accuracy_score(y_test, y_pred_class)
print(f"The accuracy of the model is: {accuracy}")

The Mean Squared Error (MSE) on test set: 0.14885532214584368
The accuracy of the model is: 0.79


In [21]:
df_2.head()

,track_id_encoded,track_artist,playlist_subgenre,danceability,energy,loudness,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_minutes,affinity_score,avg_bpm,age,avg_cadence,avg_pace,likes_song
0,215,6098,18,0.675,0.868,-6.360,0.0392,0.00107,0.715000,0.3740,0.0376,128.007,5.07,0.287000,165.0,16.0,83.0,6.27,0
1,2080,2462,9,0.605,0.584,-7.043,0.1970,0.28600,0.000000,0.2350,0.4640,125.802,4.09,0.504277,165.0,16.0,83.0,6.27,1
2,521,1083,7,0.673,0.814,-5.411,0.0367,0.01330,0.002930,0.3430,0.7280,147.193,4.20,0.395855,165.0,16.0,83.0,6.27,0
3,887,5252,11,0.557,0.979,-4.254,0.0489,0.00130,0.834000,0.0483,0.8560,127.998,4.81,0.285107,165.0,16.0,83.0,6.27,0
4,953,5066,0,0.587,0.977,-3.589,0.0711,0.03530,0.000005,0.5360,0.5190,146.518,4.46,0.292775,165.0,16.0,83.0,6.27,0


# Generating real data: 
Per user, the input will be: 
- avg_bpm	
- age
- avg_cadence
- avg_pace



In [22]:
#  MOCK VALUES !!!
avg_bpm = 180
age = 22
avg_cadence = 160
avg_pace = 6.5

user_traits = {'avg_bpm':avg_bpm, 'age': age, 'avg_cadence': avg_cadence, 'avg_pace': avg_pace}

In [23]:
df_2 = df_2[['avg_bpm', 'age', 'avg_cadence', 'avg_pace'] + [col for col in df_2.columns if col not in ['avg_bpm', 'age', 'avg_cadence', 'avg_pace']]]
df_2.drop(['likes_song', 'affinity_score'], axis=1, inplace=True)

### Imputting the user's values into the song df

In [24]:
input_columns = ['avg_bpm', 'age', 'avg_cadence', 'avg_pace']
for col in input_columns:
    df_2[col] = user_traits[col]
    
df_2.head()

,avg_bpm,age,avg_cadence,avg_pace,track_id_encoded,track_artist,playlist_subgenre,danceability,energy,loudness,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_minutes
0,180,22,160,6.5,215,6098,18,0.675,0.868,-6.360,0.0392,0.00107,0.715000,0.3740,0.0376,128.007,5.07
1,180,22,160,6.5,2080,2462,9,0.605,0.584,-7.043,0.1970,0.28600,0.000000,0.2350,0.4640,125.802,4.09
2,180,22,160,6.5,521,1083,7,0.673,0.814,-5.411,0.0367,0.01330,0.002930,0.3430,0.7280,147.193,4.20
3,180,22,160,6.5,887,5252,11,0.557,0.979,-4.254,0.0489,0.00130,0.834000,0.0483,0.8560,127.998,4.81
4,180,22,160,6.5,953,5066,0,0.587,0.977,-3.589,0.0711,0.03530,0.000005,0.5360,0.5190,146.518,4.46


### Predicting for that specific user

In [25]:
y_pred = gbm.predict(df_2, num_iteration=gbm.best_iteration, predict_disable_shape_check=True)
y_pred_class = [1 if prob >= 0.5 else 0 for prob in y_pred]

In [26]:
df_2['y_pred'] = y_pred_class

In [27]:
df_track_artist_mapping = pd.read_csv('../data_prep/track_artist_mapping.csv')

# Create a mapping dictionary from the DataFrame
artist_mapping = pd.Series(df_track_artist_mapping.track_artist.values, index=df_track_artist_mapping.label).to_dict()

# Use the mapping to replace the encoded labels in 'track_artist'
df['track_artist'] = df['track_artist'].map(artist_mapping)

In [28]:
df.head()

,track_id_encoded,track_id,track_name,track_artist,track_popularity,track_album_id,track_album_name,track_album_release_date,playlist_name,playlist_id,...,intensity_zone,intensity_zone_encoded,cluster,affinity_score,gender,avg_bpm,age,avg_cadence,avg_pace,likes_song
0,215,0hv6ku4431kPrjbpBBspw3,Animals - Original Mix,Martin Garrix,5,0N5QcojgdA6aP9oQ7yYG9A,1148,2013-07-01,gym melbourne house,5BqOZpVEqRDfZScvW1QUyA,...,Zone 4 (Hard),3,10,0.287000,1,165.0,16.0,83.0,6.27,0
1,2080,6uYAPqxP69zfpCbsTHXgiz,Cater 2 U,Destiny's Child,62,0b6ivSFfDs38MG7aLn9rvO,4221,2004-11-16,hip hop rnb,0Ar0Ng9DlAWZtSPBvOQgOa,...,Zone 3 (Moderate),2,14,0.504277,1,165.0,16.0,83.0,6.27,1
2,521,1gzIbdFnGJ226LTl0Cn2SX,White Wedding - Pt. 1,Billy Idol,68,5ebh0MxkqELmk50xHj2b4k,2079,1982-01-01,hard rock cafe,55ybnG2z8rpTIerRfnCsw6,...,Zone 5 (Maximum),4,0,0.395855,1,165.0,16.0,83.0,6.27,0
3,887,2vX03yHcVtFUg2ZMuzhLhS,Outlaw - Original Mix,La Fuente,0,7jV5akfXEXsoNsqrekDCyX,12116,2014-04-21,latin village,3JcJCsvBnRsA6oc7eUYDSB,...,Zone 4 (Hard),3,10,0.285107,1,165.0,16.0,83.0,6.27,0
4,953,38LrhW3eQSt3cK88dkvl5e,Misty Night Cruising,Kiyotaka Sugiyama,33,24sd1ZTrjjabj0A54XK0RP,11054,1984-12-21,city pop シティーポップ,3j2osvmecEao5nmo9jZ5df,...,Zone 5 (Maximum),4,0,0.292775,1,165.0,16.0,83.0,6.27,0


### Retrieving the song name and the pred value

In [29]:
merged_df = df.merge(df_2[['track_id_encoded', 'y_pred']], how='left', on='track_id_encoded', suffixes=('', '_y'))
merged_df.drop(['likes_song', 'affinity_score', 'gender'], axis=1, inplace=True)
merged_df = merged_df.loc[:, ~merged_df.columns.duplicated()]
merged_df[['track_name', 'y_pred','track_name' ]]



,track_name,y_pred,track_name
0,Animals - Original Mix,0,Animals - Original Mix
1,Cater 2 U,1,Cater 2 U
2,White Wedding - Pt. 1,1,White Wedding - Pt. 1
3,Outlaw - Original Mix,0,Outlaw - Original Mix
4,Misty Night Cruising,0,Misty Night Cruising
...,...,...,...
2769,Keep On,1,Keep On
2770,Keep On,1,Keep On
2771,Sad (feat. Afrojack),1,Sad (feat. Afrojack)
2772,Call The Parade,0,Call The Parade


In [39]:

# Get the track names and track artists of the songs that the model predicts the user will like
track_names_artists = merged_df.loc[merged_df['y_pred'] == 1, ['track_name', 'track_artist']]

# Get a random subset of 10 track names and artists
random_subset = track_names_artists.sample(n=10)
random_subset


,track_name,track_artist
1649,Alive,Snavs
1345,40 Graus,Zeus
2703,Complicated (feat. Kiiara),Dimitri Vegas & Like Mike
1533,Toothbrush,DNCE
1436,OUT WEST (feat. Young Thug),JACKBOYS
243,Pour Some Sugar On Me - Remastered 2017,Def Leppard
290,Don't Turn Your Love Away,Ballin Entertainment
2354,Shot Me Down (feat. Skylar Grey) - Radio Edit,David Guetta
2701,Family,The Chainsmokers
303,I Loved You (feat. Melissa Steel),Blonde


### To get real data: each user has to corroborar que el prediction era real, y si no hay que cambiarlo. 

In [41]:
def get_user_feedback(recommended_songs_df):
    liked_songs = []
    for _, song in recommended_songs_df.iterrows():
        # Assume a simple Yes/No prompt for the user to indicate if they like the song
        feedback = input(f"Do you like this song? {song['track_name']} by {song['track_artist']} (Yes/No): ").lower()
        if feedback == 'yes':
            liked_songs.append(song)
    
    # Convert the list of liked songs to a DataFrame
    df_liked_songs = pd.DataFrame(liked_songs)
    
    return df_liked_songs

# Use the function with the random_subset DataFrame
df_liked_songs = get_user_feedback(random_subset)

In [40]:
df_liked_songs.head()

""


In [ ]:
# import requests

# def get_spotify_top_data(access_token, data_type='tracks'):
#     url = f'https://api.spotify.com/v1/me/top/{data_type}'
#     headers = {'Authorization': f'Bearer {access_token}'}
#     response = requests.get(url, headers=headers)
#     if response.status_code == 200:
#         return [item['id'] for item in response.json()['items']]
#     else:
#         print(f"Error fetching top {data_type}: {response.status_code}")
#         return []

# # Fetch top tracks and artists
# top_tracks = get_spotify_top_data(access_token, 'tracks')
# top_artists = get_spotify_top_data(access_token, 'artists')

# # Assuming model_recommendations is a list of track IDs from your model
# model_recommendations = [...]

# # Find intersection
# liked_songs = set(top_tracks) & set(df_liked_songs)